In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Reading from socket")
    .master("local[*]").getOrCreate()
)

In [3]:
df_raw = spark.readStream.format('socket').option("host","localhost").option("port","9999").load()

In [4]:
# Split the line into words
from pyspark.sql.functions import split

df_words = df_raw.withColumn("words", split("value", " "))

In [5]:
# Explode the list of words
from pyspark.sql.functions import explode

df_explode = df_words.withColumn("word", explode("words")).drop("value", "words")

In [6]:
# Aggregate the words to generate count
from pyspark.sql.functions import count, lit

df_agg = df_explode.groupBy("word").agg(count(lit(1)).alias("cnt"))

In [ ]:
df_agg.writeStream.format("console").outputMode("complete").start().awaitTermination()